# Database Admin 101 - Lab

## Introduction 

In this lab, you'll go through the process of designing and creating a database. From there, you'll begin to populate this table with mock data provided to you.

## Objectives

You will be able to:

* Use knowledge of the structure of databases to create a database and populate it

## The Scenario

You are looking to design a database for a school that will house various information from student grades to contact information, class roster lists and attendance. First, think of how you would design such a database. What tables would you include? What columns would each table have? What would be the primary means to join said tables?

## Creating the Database

Now that you've put a little thought into how you might design your database, it's time to go ahead and create it! Start by import the necessary packages. Then, create a database called **school.sqlite**.

In [23]:
# Import necessary packages
import pandas as pd
import numpy as np
import sqlite3

In [39]:
# Create the database school.sqlite 
conn = sqlite3.connect('school.sqlite')
cur = conn.cursor()


## Create a Table for Contact Information

Create a table called contactInfo to house contact information for both students and staff. Be sure to include columns for first name, last name, role (student/staff), telephone number, street, city, state, and zipcode. Be sure to also create a primary key for the table. 

In [40]:
# Your code here
strSQL = """
    CREATE TABLE contactInfo (
        userId INTEGER PRIMARY KEY,
        firstName TEXT,
        lastName  TEXT,
        role TEXT,
        telephone INTEGER,
        street TEXT,
        city TEXT,
        state TEXT,
        zipcode TEXT
    )
"""
cur.execute(strSQL)

## Populate the Table

Below, code is provided for you in order to load a list of dictionaries. Briefly examine the list. Each dictionary in the list will serve as an entry for your contact info table. Once you've briefly investigated the structure of this data, write a for loop to iterate through the list and create an entry in your table for each person's contact info.

In [41]:
# Load the list of dictionaries; just run this cell
import pickle

with open('contact_list.pickle', 'rb') as f:
    contacts = pickle.load(f)

print(contacts[0])

{'firstName': 'Christine', 'lastName': 'Holden', 'role': 'staff', 'telephone ': 2035687697, 'street': '1672 Whitman Court', 'city': 'Stamford', 'state': 'CT', 'zipcode ': '06995'}


In [42]:
# Iterate over the contact list and populate the contactInfo table here
for contact in contacts:
    strColNames = '('
    strColValues = '('
    for k,v in contact.items():
        k = k.strip()
        strColNames += (k+',')
        if k != 'telephone':
            strColValues += ('\''+v+'\',')
        else:
            strColValues += (str(v)+',')
    strColValues = strColValues[:-1] + ')'
    strColNames = strColNames[:-1] + ')'
    cur.execute(f'''INSERT INTO contactInfo {strColNames} 
                  VALUES {strColValues};
            ''')

**Query the Table to Ensure it is populated**

In [43]:
# Your code here 
df = pd.read_sql("""SELECT * FROM contactInfo""", conn)
df

,contactId,firstName,lastName,role,telephone,street,city,state,zipcode
0,1,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995
1,2,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
2,3,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,4,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,5,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
5,6,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
6,7,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
7,8,Ed,Lyman,student,5179695576,3478 Be Sreet,Lansing,MI,48933


## Commit Your Changes to the Database

Persist your changes by committing them to the database.

In [47]:
# Your code here
conn.commit()
conn.close()

In [48]:
conn1 = sqlite3.connect('school.sqlite')
df = pd.read_sql("""SELECT * FROM contactInfo""", conn1)
df

,contactId,firstName,lastName,role,telephone,street,city,state,zipcode
0,1,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995
1,2,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
2,3,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,4,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,5,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
5,6,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
6,7,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
7,8,Ed,Lyman,student,5179695576,3478 Be Sreet,Lansing,MI,48933


In [49]:
conn1.close()

## Create a Table for Student Grades

Create a new table in the database called "grades". In the table, include the following fields: userId, courseId, grade.

** This problem is a bit more tricky and will require a dual key. (A nuance you have yet to see.)
Here's how to do that:

```SQL
CREATE TABLE table_name(
   column_1 INTEGER NOT NULL,
   column_2 INTEGER NOT NULL,
   ...
   PRIMARY KEY(column_1,column_2,...)
);
```

In [55]:
# Create the grades table
conn = sqlite3.connect('school.sqlite')
cur = conn.cursor()
strSQL = """
CREATE TABLE grades(
    userId INTEGER NOT NULL,
    courseId INTEGER NOT NULL,
    grade INTEGER NOT NULL,
    PRIMARY KEY(userId,courseId));
"""
cur.execute(strSQL)

## Remove Duplicate Entries

An analyst just realized that there is a duplicate entry in the contactInfo table! Find and remove it.

In [60]:
# Find the duplicate entry
strSQL = """
    SELECT lastName, firstName, COUNT(contactId)
    FROM contactInfo
    GROUP BY lastName, firstName
    HAVING COUNT(contactId) > 1
"""
df = pd.read_sql(strSQL,conn)
df

,lastName,firstName,COUNT(contactId)
0,Evans,Jane,2


In [61]:
strSQL = """
    SELECT contactId
    FROM contactInfo
    WHERE lastName = 'Evans' AND firstName = 'Jane'
"""
cur.execute(strSQL).fetchall()

[(5,), (6,)]

In [62]:
# Delete the duplicate entry
strSQL = """
DELETE FROM contactInfo where contactId = 6;
"""
cur.execute(strSQL)

In [64]:
# Check that the duplicate entry was removed
df = pd.read_sql("""SELECT * FROM contactInfo""",conn)
df

,contactId,firstName,lastName,role,telephone,street,city,state,zipcode
0,1,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995
1,2,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
2,3,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,4,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,5,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
5,7,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
6,8,Ed,Lyman,student,5179695576,3478 Be Sreet,Lansing,MI,48933


## Updating an Address

Ed Lyman just moved to `2910 Simpson Avenue York, PA 17403`. Update his address accordingly.

In [65]:
# Update Ed's address
strSQL = """
    UPDATE contactInfo
    SET street = '2910 Simpson Avenue', city = 'York', state = 'PA', zipcode = '17403'
    WHERE lastName = 'Lyman' AND firstName = 'Ed'
"""
cur.execute(strSQL)

In [66]:
# Query the database to ensure the change was made
df = pd.read_sql("""SELECT * FROM contactInfo""",conn)
df

,contactId,firstName,lastName,role,telephone,street,city,state,zipcode
0,1,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995
1,2,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
2,3,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,4,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,5,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
5,7,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
6,8,Ed,Lyman,student,5179695576,2910 Simpson Avenue,York,PA,17403


## Commit Your Changes to the Database

Once again, persist your changes by committing them to the database.

In [67]:
# Your code here
conn.commit()
conn.close()

## Summary

While there's certainly more to do with setting up and managing this database, you got a taste for creating, populating, and maintaining databases! Feel free to continue fleshing out this exercise for more practice. 